In [13]:
# import sys # 即添加包名的搜索路径
# sys.path.append("/root/autodl-tmp/moatless-tree-search-main/moatless")

In [1]:
from dotenv import load_dotenv
import os

# 加载 .env 文件
load_dotenv()

True

In [2]:
import json
from typing import Optional, Dict, Any, List, Callable, Union
from textwrap import dedent

from moatless.benchmark.utils import get_moatless_instance
from moatless.completion.model import StructuredOutput, Completion
from moatless.completion.completion import CompletionModel, CompletionResponse

/root/miniconda3/lib/python3.12/site-packages/pydantic/_internal/_config.py:345: UserWarning: Valid config keys have changed in V2:
* 'fields' has been removed
  warnings.warn(message, UserWarning)
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /root/miniconda3/lib/python3.12/site-
[nltk_data]     packages/llama_index/core/_static/nltk_cache...
[nltk_data]   Package punkt_tab is already up-to-date!
/root/miniconda3/lib/python3.12/site-packages/pydantic/_internal/_fields.py:192: UserWarning: Field name "name" in "FeedbackResponse" shadows an attribute in parent "StructuredOutput"
  warnings.warn(


# CompletionModel

In [3]:
# litellm对于托管在openai proxy的模型必须在名字前加 'openai/'
completion_model = CompletionModel(model="openai/moonshot-v1-32k", model_base_url=os.getenv("CUSTOM_LLM_API_BASE"), model_api_key=os.getenv("CUSTOM_LLM_API_KEY"), temperature=0.7)
# completion_model = CompletionModel(model="deepseek/deepseek-chat", temperature=0.7)
completion_model

CompletionModel(model='openai/moonshot-v1-32k', temperature=0.7, max_tokens=2000, timeout=120.0, model_base_url='https://api.moonshot.cn/v1', model_api_key='sk-ZfLc3KIKE88O45O53KLEmmwii5wB2ZPUA9CGvNST3ohaGwNN', response_format=None, stop_words=None, metadata=None, thoughts_in_action=False)

In [40]:
messages = [
    {"role": "user", "content": "请告诉我 1+1=?"}
]

# completion_model._litellm_base_completion(messages=messages)

In [180]:
system_prompt = dedent(
    """
    你是陈思霖的助手，请提供准确且简洁的回答!
    """
)

response_model = StructuredOutput

out = completion_model.create_completion(messages=messages, system_prompt=system_prompt, response_model=response_model)
out

CompletionResponse(structured_outputs=[StructuredOutput()], text_response=None, completion=Completion(model='deepseek/deepseek-chat', input=[{'role': 'system', 'content': '\n你是陈思霖的助手，请提供准确且简洁的回答!\n\n# Response format\n        You must respond with only a JSON object that match the following json_schema:\n\n\n        {\n  "properties": {},\n  "title": "StructuredOutput",\n  "type": "object"\n}\n\n        Make sure to return an instance of the JSON, not the schema itself.'}, {'role': 'user', 'content': '请告诉我 1+1=?'}, {'role': 'assistant', 'content': '{"answer": 2}'}], response={'id': '20ff6dff-ec33-4c8f-9aa5-b421d53bce99', 'created': 1740362436, 'model': 'deepseek/deepseek-chat', 'object': 'chat.completion', 'system_fingerprint': 'fp_3a5770e1b4_prod', 'choices': [{'finish_reason': 'stop', 'index': 0, 'message': {'content': '{"answer": 2}', 'role': 'assistant', 'tool_calls': None, 'function_call': None}}], 'usage': {'completion_tokens': 10, 'prompt_tokens': 96, 'total_tokens': 106, 'compl

## model内部

In [25]:
system_prompt = dedent(
    """
    你是陈思霖的助手，请提供准确且简洁的回答!
    """
)

response_model = StructuredOutput

system_prompt += dedent(f"""\n# Response format
You must respond with only a JSON object that match the following json_schema:\n

{json.dumps(response_model.model_json_schema(), indent=2, ensure_ascii=False)}

Make sure to return an instance of the JSON, not the schema itself.""")

messages.insert(0, {"role": "system", "content": system_prompt})

print(messages)

[{'role': 'system', 'content': '\n你是陈思霖的助手，请提供准确且简洁的回答!\n\n# Response format\nYou must respond with only a JSON object that match the following json_schema:\n\n\n{\n  "properties": {},\n  "title": "StructuredOutput",\n  "type": "object"\n}\n\nMake sure to return an instance of the JSON, not the schema itself.'}, {'role': 'system', 'content': '\n你是陈思霖的助手，请提供准确且简洁的回答!\n\n# Response format\nYou must respond with only a JSON object that match the following json_schema:\n\n\n{\n  "properties": {},\n  "title": "StructuredOutput",\n  "type": "object"\n}\n\nMake sure to return an instance of the JSON, not the schema itself.'}, {'role': 'user', 'content': '请告诉我 1+1=?'}, {'role': 'assistant', 'content': '{}'}]


In [17]:
completion_response = completion_model._litellm_base_completion(
                    messages=messages, response_format={"type": "json_object"}
                )

In [20]:
messages

[{'role': 'system',
  'content': '\n你是陈思霖的助手，请提供准确且简洁的回答!\n\n# Response format\nYou must respond with only a JSON object that match the following json_schema:\n\n\n{\n  "properties": {},\n  "title": "StructuredOutput",\n  "type": "object"\n}\n\nMake sure to return an instance of the JSON, not the schema itself.'},
 {'role': 'user', 'content': '请告诉我 1+1=?'}]

In [18]:
assistant_message = completion_response.choices[0].message.content
assistant_message

'{}'

In [21]:
messages.append({"role": "assistant", "content": assistant_message})
messages

[{'role': 'system',
  'content': '\n你是陈思霖的助手，请提供准确且简洁的回答!\n\n# Response format\nYou must respond with only a JSON object that match the following json_schema:\n\n\n{\n  "properties": {},\n  "title": "StructuredOutput",\n  "type": "object"\n}\n\nMake sure to return an instance of the JSON, not the schema itself.'},
 {'role': 'user', 'content': '请告诉我 1+1=?'},
 {'role': 'assistant', 'content': '{}'}]

In [22]:
response = response_model.model_validate_json(assistant_message)
response

StructuredOutput()

In [23]:
from moatless.completion.model import Completion

completion = Completion.from_llm_completion(
    input_messages=messages,
    completion_response=completion_response,
    model="deepseek/deepseek-chat",
)
completion

Completion(model='deepseek/deepseek-chat', input=[{'role': 'system', 'content': '\n你是陈思霖的助手，请提供准确且简洁的回答!\n\n# Response format\nYou must respond with only a JSON object that match the following json_schema:\n\n\n{\n  "properties": {},\n  "title": "StructuredOutput",\n  "type": "object"\n}\n\nMake sure to return an instance of the JSON, not the schema itself.'}, {'role': 'user', 'content': '请告诉我 1+1=?'}, {'role': 'assistant', 'content': '{}'}], response={'id': '3505b491-4bb1-4678-b6fe-d2305e68a873', 'created': 1740361131, 'model': 'deepseek/deepseek-chat', 'object': 'chat.completion', 'system_fingerprint': 'fp_3a5770e1b4_prod', 'choices': [{'finish_reason': 'stop', 'index': 0, 'message': {'content': '{}', 'role': 'assistant', 'tool_calls': None, 'function_call': None}}], 'usage': {'completion_tokens': 5, 'prompt_tokens': 93, 'total_tokens': 98, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 64}, 'prompt_cache_hit_tokens': 64, 'prompt_c

In [43]:
CompletionResponse.create(
                        output=response, completion=completion
                    )

CompletionResponse(structured_outputs=[StructuredOutput()], text_response=None, completion=Completion(model='openai/moonshot-v1-8k', input=[{'role': 'system', 'content': '\n你是陈思霖的助手，请提供准确且简洁的回答，每次回答前要加上陈思霖大人！\n\n# Response format\nYou must respond with only a JSON object that match the following json_schema:\n\n\n{\n  "properties": {},\n  "title": "StructuredOutput",\n  "type": "object"\n}\n\nMake sure to return an instance of the JSON, not the schema itself.'}, {'role': 'user', 'content': '请告诉我 1+1=?'}, {'role': 'assistant', 'content': '{\n  "properties": {\n    "answer": "1+1=2"\n  },\n  "title": "StructuredOutput",\n  "type": "object"\n}'}], response={'id': 'chatcmpl-67b6a025457a3e1f37907f39', 'created': 1740021797, 'model': 'moonshot-v1-8k', 'object': 'chat.completion', 'system_fingerprint': None, 'choices': [{'finish_reason': 'stop', 'index': 0, 'message': {'content': '{\n  "properties": {\n    "answer": "1+1=2"\n  },\n  "title": "StructuredOutput",\n  "type": "object"\n}', 'role'

# Environment

In [4]:
from moatless.benchmark.swebench import create_repository
from moatless.index import CodeIndex
from moatless.file_context import FileContext
from moatless.selector import BestFirstSelector
from moatless.value_function.base import ValueFunction

In [5]:
instance = get_moatless_instance(split='verified', instance_id="django__django-11880")  # 获得的instance是本地下载下来有点删改属性的swe-bench

In [6]:
repository = create_repository(instance)

In [7]:
repository

GitRepository(repo_path='/root/autodl-tmp/moatless-tree-search-main/tmp/repos/swe-bench_django__django-11880', repo_url=None, generate_commit_message=False, completion=None, current_commit='06909fe084f87a65459a83bd69d7cdbe4fce9a7c', current_diff=None, initial_commit='06909fe084f87a65459a83bd69d7cdbe4fce9a7c')

In [8]:
code_index = CodeIndex.from_index_name(
    instance["instance_id"], file_repo=repository
)

In [9]:
code_index.find_function(class_name='Apps', function_name='populate')

SearchCodeResponse(message='Found 1 functions.', hits=[SearchCodeHit(file_path='django/apps/registry.py', spans=[SpanHit(span_id='Apps.populate', rank=0, tokens=386)])])

In [11]:
code_index.find_function(function_name='populate')

SearchCodeResponse(message='Found 2 functions.', hits=[SearchCodeHit(file_path='django/apps/registry.py', spans=[SpanHit(span_id='Apps.populate', rank=0, tokens=386)]), SearchCodeHit(file_path='django/db/models/query.py', spans=[SpanHit(span_id='RelatedPopulator.populate', rank=1, tokens=111)])])

In [37]:
instance['problem_statement'][:50]

'Set default FILE_UPLOAD_PERMISSION to 0o644.\nDescr'

In [10]:
code_index.semantic_search(query=instance['problem_statement'][:50])

SearchCodeResponse(message='Found 101 code spans.', hits=[SearchCodeHit(file_path='tests/file_storage/tests.py', spans=[SpanHit(span_id='FileStoragePermissions', rank=0, tokens=27), SpanHit(span_id='FileStoragePermissions.setUp', rank=0, tokens=33), SpanHit(span_id='FileStoragePermissions.tearDown', rank=0, tokens=19), SpanHit(span_id='FileStoragePermissions.test_file_upload_default_permissions', rank=0, tokens=71), SpanHit(span_id='FileStoragePermissions.test_file_upload_directory_default_permissions', rank=0, tokens=81), SpanHit(span_id='FileStoragePermissions.test_file_upload_directory_permissions', rank=0, tokens=78), SpanHit(span_id='FileStoragePermissions.test_file_upload_permissions', rank=0, tokens=70), SpanHit(span_id='FileStorageTests.test_setting_changed', rank=3, tokens=308)]), SearchCodeHit(file_path='tests/staticfiles_tests/test_storage.py', spans=[SpanHit(span_id='TestStaticFilePermissions', rank=1, tokens=54), SpanHit(span_id='TestStaticFilePermissions.run_collectstatic

In [10]:
file_context = FileContext(repo=repository)

In [61]:
# 这会增加_files里面的
context_file = file_context.get_context_file(file_path='/root/autodl-tmp/moatless-tree-search-main/tmp/repos/swe-bench_django__django-10914/django/apps/registry.py')

In [146]:
file_context._files

{}

In [63]:
context_file._add_import_span()

In [64]:
context_file.spans   # 在FileContext和ContextFile中，spans并不会直接初始化，应该是在后面初始化，只有import的span有函数初始化

[ContextSpan(span_id='imports', start_line=None, end_line=None, tokens=None, pinned=True)]

In [11]:
selector = BestFirstSelector()

# Coding Agent

In [12]:
from moatless.actions import FindClass, FindFunction, FindCodeSnippet, SemanticSearch, ViewCode, Finish, Reject, RunTests, StringReplace, CreateFile
from moatless.agent.code_agent import CodingAgent, create_edit_code_actions
from moatless.agent.code_prompts import *
from moatless.search_tree import SearchTree
from moatless.completion.completion import (
    LLMResponseFormat,
    CompletionModel,
)
from moatless.schema import MessageHistoryType
from moatless.message_history import MessageHistoryGenerator

In [13]:
completion_model.response_format = LLMResponseFormat.TOOLS
completion_model

CompletionModel(model='openai/moonshot-v1-32k', temperature=0.7, max_tokens=2000, timeout=120.0, model_base_url='https://api.moonshot.cn/v1', model_api_key='sk-ZfLc3KIKE88O45O53KLEmmwii5wB2ZPUA9CGvNST3ohaGwNN', response_format=<LLMResponseFormat.TOOLS: 'tool_call'>, stop_words=None, metadata=None, thoughts_in_action=False)

In [14]:
actions = [
    FindClass(completion_model=completion_model, code_index=code_index, repository=repository),
    FindFunction(completion_model=completion_model, code_index=code_index, repository=repository),
    FindCodeSnippet(completion_model=completion_model, code_index=code_index, repository=repository),
    SemanticSearch(completion_model=completion_model, code_index=code_index, repository=repository),
    ViewCode(repository=repository),
    StringReplace(repository=repository, code_index=code_index),
    CreateFile(repository=repository, code_index=code_index),
    RunTests(repository=repository, code_index=code_index),
    Finish(),
    Reject()
]

system_prompt = AGENT_ROLE
if completion_model.response_format == LLMResponseFormat.REACT:
    system_prompt += REACT_CORE_OPERATION_RULES
elif completion_model.response_format == LLMResponseFormat.TOOLS:
    system_prompt += REACT_GUIDELINES
workflow_prompt = generate_workflow_prompt(actions, False)
system_prompt += workflow_prompt + generate_guideline_prompt(False) + ADDITIONAL_NOTES
print(system_prompt)

You are an autonomous AI assistant with superior programming skills. As you're working autonomously, 
you cannot communicate with the user but must rely on information you can get from the available functions.
# Action and ReAct Guidelines

1. **Analysis First**
   - Review all previous actions and their observations
   - Understand what has been done and what information you have

2. **Document Your Thoughts**
   - ALWAYS write your reasoning in `<thoughts>` tags before any action
   - Explain what you learned from previous observations
   - Justify why you're choosing the next action
   - Describe what you expect to learn/achieve

3. **Single Action Execution**
   - Run ONLY ONE action at a time
   - Choose from the available functions
   - Never try to execute multiple actions at once

4. **Wait and Observe**
   - After executing an action, STOP
   - Wait for the observation (result) to be returned
   - Do not plan or execute any further actions until you receive the observation

# 

In [15]:
agent = CodingAgent(system_prompt=system_prompt, actions=actions, completion=completion_model)
# # 我认为应该是下面这种初始化，用的是内部的prompt而不是手动system_prompt，但是测试的时候是用了上面的初始化以及SIMPLE_CODE_PROMPT
# agent = CodingAgent.create(repository=repository, completion_model=completion_model)
# agent.actions = actions    # if not， 它内部的action没有code index，也没有repository

In [16]:
message_history_type = MessageHistoryType.MESSAGES
message_history_type

<MessageHistoryType.MESSAGES: 'messages'>

In [17]:
message_generator = MessageHistoryGenerator(
    message_history_type=message_history_type,
    include_file_context=True,
    thoughts_in_action=False,
)

In [18]:
action_completion_format = completion_model.response_format

In [19]:
action_completion_model = completion_model.clone(
    response_format=action_completion_format
)

In [20]:
action_completion_model

ToolCallCompletionModel(model='openai/moonshot-v1-32k', temperature=0.7, max_tokens=2000, timeout=120.0, model_base_url='https://api.moonshot.cn/v1', model_api_key='sk-ZfLc3KIKE88O45O53KLEmmwii5wB2ZPUA9CGvNST3ohaGwNN', response_format=<LLMResponseFormat.TOOLS: 'tool_call'>, stop_words=None, metadata=None, thoughts_in_action=False)

In [26]:
# actions = create_edit_code_actions(
#     repository=repository,
#     code_index=code_index,
#     completion_model=action_completion_model,
# )
# action_type = "standard edit code actions"
# use_few_shots = True

In [111]:
# actions

[SemanticSearch(max_search_tokens=2000, max_identify_tokens=8000, max_identify_prompt_tokens=16000, max_hits=10, completion_model=ToolCallCompletionModel(model='deepseek/deepseek-chat', temperature=0.7, max_tokens=2000, timeout=120.0, model_base_url=None, model_api_key=None, response_format=<LLMResponseFormat.TOOLS: 'tool_call'>, stop_words=None, metadata=None, thoughts_in_action=False)),
 FindClass(max_search_tokens=2000, max_identify_tokens=8000, max_identify_prompt_tokens=16000, max_hits=10, completion_model=ToolCallCompletionModel(model='deepseek/deepseek-chat', temperature=0.7, max_tokens=2000, timeout=120.0, model_base_url=None, model_api_key=None, response_format=<LLMResponseFormat.TOOLS: 'tool_call'>, stop_words=None, metadata=None, thoughts_in_action=False)),
 FindFunction(max_search_tokens=2000, max_identify_tokens=8000, max_identify_prompt_tokens=16000, max_hits=10, completion_model=ToolCallCompletionModel(model='deepseek/deepseek-chat', temperature=0.7, max_tokens=2000, tim

In [115]:
# workflow_prompt = generate_workflow_prompt(actions, False)
# print(workflow_prompt)


# Workflow Overview

1. **Understand the Task**
  * **Review the Task:** Carefully read the task provided in <task>.
  * **Identify Code to Change:** Analyze the task to determine which parts of the codebase need to be changed.
  * **Identify Necessary Context:** Determine what additional parts of the codebase are needed to understand how to implement the changes. Consider dependencies, related components, and any code that interacts with the affected areas.

2. **Locate Code**
  * **Primary Method - Search Functions:** Use these to find relevant code:
      * SemanticSearch - Search code by semantic meaning and natural language description
      * FindClass - Search for class definitions by class name
      * FindFunction - Search for function definitions by function name
      * FindCodeSnippet - Search for specific code patterns or text
  * **Secondary Method - ViewCode:** Only use when you need to see:
      * Additional context not returned by searches
      * Specific line range

In [29]:
# system_prompt = AGENT_ROLE
# if completion_model.response_format == LLMResponseFormat.REACT:
#     system_prompt += REACT_CORE_OPERATION_RULES
# elif completion_model.response_format == LLMResponseFormat.TOOLS:
#     system_prompt += REACT_GUIDELINES
# print(system_prompt)

You are an autonomous AI assistant with superior programming skills. As you're working autonomously, 
you cannot communicate with the user but must rely on information you can get from the available functions.
# Action and ReAct Guidelines

1. **Analysis First**
   - Review all previous actions and their observations
   - Understand what has been done and what information you have

2. **Document Your Thoughts**
   - ALWAYS write your reasoning in `<thoughts>` tags before any action
   - Explain what you learned from previous observations
   - Justify why you're choosing the next action
   - Describe what you expect to learn/achieve

3. **Single Action Execution**
   - Run ONLY ONE action at a time
   - Choose from the available functions
   - Never try to execute multiple actions at once

4. **Wait and Observe**
   - After executing an action, STOP
   - Wait for the observation (result) to be returned
   - Do not plan or execute any further actions until you receive the observation



In [30]:
# system_prompt += workflow_prompt + generate_guideline_prompt(False) + ADDITIONAL_NOTES
# print(system_prompt)

You are an autonomous AI assistant with superior programming skills. As you're working autonomously, 
you cannot communicate with the user but must rely on information you can get from the available functions.
# Action and ReAct Guidelines

1. **Analysis First**
   - Review all previous actions and their observations
   - Understand what has been done and what information you have

2. **Document Your Thoughts**
   - ALWAYS write your reasoning in `<thoughts>` tags before any action
   - Explain what you learned from previous observations
   - Justify why you're choosing the next action
   - Describe what you expect to learn/achieve

3. **Single Action Execution**
   - Run ONLY ONE action at a time
   - Choose from the available functions
   - Never try to execute multiple actions at once

4. **Wait and Observe**
   - After executing an action, STOP
   - Wait for the observation (result) to be returned
   - Do not plan or execute any further actions until you receive the observation

# 

In [22]:
code_index is not None

True

# Search Tree

In [21]:
import random
from typing import List

from pydantic import BaseModel, Field

from moatless.agent.settings import AgentSettings
from moatless.node import Node, ActionStep
from moatless.expander import Expander
persist_path = "/root/autodl-tmp/moatless-tree-search-main/tmp/trajectory/django__django-10914-trajectory.json"

# search_tree = SearchTree.create(
#     message=instance["problem_statement"],
#     agent=agent,
#     file_context=file_context,
#     selector=selector,
#     max_iterations=100,
#     max_expansions=3,
#     max_depth=25,
#     persist_path=persist_path,
# )

# node = search_tree.run_search()
# print(node.observation.message)

In [22]:
root = Node(
    node_id=0,
    max_expansions=3,
    user_message=instance["problem_statement"],   # 源代码是message == ...，但我发现Node类根本没有这个属性，有设置user_message的一个叫message的方法
    file_context=file_context,
)
print(root)

node_id=0 parent=None children=[] workspace=None artifact_changes=[] user_message='Form Field’s __deepcopy__ does not (deep)copy the error messages.\nDescription\n\t\nThe __deepcopy__ method defined for the formfields (\u200bhttps://github.com/django/django/blob/146086f219d01dbb1cd8c089b5a5667e396e1cc4/django/forms/fields.py#L200) performs a shallow copy of self and does not include additional treatment for the error_messages dictionary. As a result, all copies of the same field share the same dictionary and any modification of either the dictionary or the error message itself for one formfield is immediately reflected on all other formfiels.\nThis is relevant for Forms and ModelForms that modify the error messages of their fields dynamically: while each instance of the specific form (e.g., ProfileForm) is expected to have a set of fields “sealed” away from other instances of the same ProfileForm (\u200bhttps://github.com/django/django/blob/146086f219d01dbb1cd8c089b5a5667e396e1cc4/djan

In [23]:
def _select(node: Node) -> Optional[Node]:
    """Select a node for expansion using the UCT algorithm."""
    expandable_nodes = node.get_expandable_descendants()

    if not expandable_nodes:
        print("No expandable nodes found.")
        return None

    # If we have a finished node or exceeded depth, use normal selection
    return selector.select(expandable_nodes)

In [24]:
node = _select(root)
node

Node(node_id=0, parent=None, children=[], workspace=None, artifact_changes=[], user_message='Form Field’s __deepcopy__ does not (deep)copy the error messages.\nDescription\n\t\nThe __deepcopy__ method defined for the formfields (\u200bhttps://github.com/django/django/blob/146086f219d01dbb1cd8c089b5a5667e396e1cc4/django/forms/fields.py#L200) performs a shallow copy of self and does not include additional treatment for the error_messages dictionary. As a result, all copies of the same field share the same dictionary and any modification of either the dictionary or the error message itself for one formfield is immediately reflected on all other formfiels.\nThis is relevant for Forms and ModelForms that modify the error messages of their fields dynamically: while each instance of the specific form (e.g., ProfileForm) is expected to have a set of fields “sealed” away from other instances of the same ProfileForm (\u200bhttps://github.com/django/django/blob/146086f219d01dbb1cd8c089b5a5667e396

In [25]:
unique_id=0

In [26]:
class Expander(BaseModel):
    random_settings: bool = Field(
        False, description="Whether to select agent settings randomly"
    )
    max_expansions: int = Field(
        1, description="The maximum number of children to create for each node"
    )

    agent_settings: List[AgentSettings] = Field(
        [],
        description="The settings for the agent model",
    )

    def expand(
        self, node: Node, unique_id, force_expansion: bool = False
    ) -> None | Node:
        """Handle all node expansion logic in one place"""
        if not force_expansion and node.is_fully_expanded():
            return None

        # Return the first unexecuted child if one exists
        for child in node.children:
            if not child.observation:
                print(f"Found unexecuted child {child.node_id} for node {node.node_id}")
                return child

        num_expansions = node.max_expansions or self.max_expansions
        if not force_expansion and len(node.children) >= num_expansions:
            print(f"Max expansions reached for node {node.node_id}")
            return None

        settings_to_use = self._get_agent_settings(node)

        child_node = Node(
            node_id=unique_id,
            parent=node,
            file_context=node.file_context.clone() if node.file_context else None,
            max_expansions=self.max_expansions,
            agent_settings=settings_to_use[0] if settings_to_use else None,
        )

        node.add_child(child_node)

        return child_node

    def _get_agent_settings(self, node: Node) -> List[AgentSettings]:
        """Get agent settings for a single expansion."""
        if not self.agent_settings:
            return []

        if self.random_settings:
            used_settings = {
                child.agent_settings
                for child in node.children
                if child.agent_settings is not None
            }

            available_settings = [
                setting
                for setting in self.agent_settings
                if setting not in used_settings
            ]

            settings_pool = available_settings or self.agent_settings
            return [random.choice(settings_pool)]
        else:
            num_children = len(node.children)
            return [self.agent_settings[num_children % len(self.agent_settings)]]

    def _generate_unique_id(self, node: Node):
        return len(node.get_root().get_all_nodes())

In [27]:
expander = Expander(max_expansions=3)

In [28]:
def _expand(node: Node, force_expansion: bool = False) -> Node:
    """Expand the node and return a child node."""

    # Check if any action step was not executed, if so return the node
    if node.action_steps and node.has_unexecuted_actions():
        print(f"Returning Node{node.node_id} with unexecuted actions")
        return node

    global unique_id
    child_node = expander.expand(node, unique_id := 1, force_expansion)   # 原代码的流程都没有输入expander，默认值是None

    if not node.action_steps and not node.assistant_message:
        child_node.user_message = "You're an autonomous AI agent that must respond with one of the provided functions"

    # # Only add feedback if this is the second expansion from this node
    # if self.feedback_generator and len(node.children) >= 2:
    #     child_node.feedback_data = self.feedback_generator.generate_feedback(
    #         child_node,
    #         self.agent.actions,
    #     )

    # self.log(
    #     logger.info, f"Expanded Node{node.node_id} to new Node{child_node.node_id}"
    # )
    print(f"Expanded Node{node.node_id} to new Node{child_node.node_id}")
    return child_node

In [29]:
new_node = _expand(node)
new_node

Expanded Node0 to new Node1


Node(node_id=1, parent=Node(node_id=0, parent=None, children=[Node(node_id=1, parent=Node(node_id=0, parent=None, children=[...], workspace=None, artifact_changes=[], user_message='Form Field’s __deepcopy__ does not (deep)copy the error messages.\nDescription\n\t\nThe __deepcopy__ method defined for the formfields (\u200bhttps://github.com/django/django/blob/146086f219d01dbb1cd8c089b5a5667e396e1cc4/django/forms/fields.py#L200) performs a shallow copy of self and does not include additional treatment for the error_messages dictionary. As a result, all copies of the same field share the same dictionary and any modification of either the dictionary or the error message itself for one formfield is immediately reflected on all other formfiels.\nThis is relevant for Forms and ModelForms that modify the error messages of their fields dynamically: while each instance of the specific form (e.g., ProfileForm) is expected to have a set of fields “sealed” away from other instances of the same Prof

## SearchTree._simulate函数内部

In [30]:
message_generator.message_history_type

<MessageHistoryType.MESSAGES: 'messages'>

In [31]:
new_node.possible_actions

[]

In [32]:
new_node.possible_actions = [action.name for action in agent.actions]
system_prompt = agent.generate_system_prompt()
action_args = [action.args_schema for action in agent.actions]

messages = message_generator.generate(new_node)
print(f"Node{new_node.node_id}: Build action with {len(messages)} messages")

Node1: Build action with 2 messages


In [33]:
messages

[{'role': 'user',
  'content': [{'type': 'text',
    'text': 'Form Field’s __deepcopy__ does not (deep)copy the error messages.\nDescription\n\t\nThe __deepcopy__ method defined for the formfields (\u200bhttps://github.com/django/django/blob/146086f219d01dbb1cd8c089b5a5667e396e1cc4/django/forms/fields.py#L200) performs a shallow copy of self and does not include additional treatment for the error_messages dictionary. As a result, all copies of the same field share the same dictionary and any modification of either the dictionary or the error message itself for one formfield is immediately reflected on all other formfiels.\nThis is relevant for Forms and ModelForms that modify the error messages of their fields dynamically: while each instance of the specific form (e.g., ProfileForm) is expected to have a set of fields “sealed” away from other instances of the same ProfileForm (\u200bhttps://github.com/django/django/blob/146086f219d01dbb1cd8c089b5a5667e396e1cc4/django/forms/forms.py#L95

In [34]:
completion_response = agent._completion.create_completion(
                messages, system_prompt=system_prompt, response_model=action_args
            )

In [35]:
completion_response.structured_outputs

[FindFunctionArgs(thoughts="To address the issue of the __deepcopy__ method in Django's form fields not properly copying the error messages, we need to locate the implementation of the __deepcopy__ method in the fields.py file and examine the code to understand how it currently handles the error_messages dictionary. This will help us determine the necessary changes to ensure that each form field has its own separate copy of the error_messages dictionary.", file_pattern=None, function_name='__deepcopy__', class_name='FormField')]

In [36]:
new_node.action_steps = [
        ActionStep(action=action)
        for action in completion_response.structured_outputs
    ]

In [37]:
new_node.assistant_message = completion_response.text_response

new_node.completions["build_action"] = completion_response.completion

In [38]:
duplicate_node = node.find_duplicate()
if duplicate_node:
    node.is_duplicate = True
    print(
        f"Node{node.node_id} is a duplicate to Node{duplicate_node.node_id}. Skipping execution."
    )
    # return

### agent._execute函数内部

In [39]:
action_step = new_node.action_steps[0]
action = agent._action_map.get(type(action_step.action))

In [40]:
action_step

ActionStep(action=FindFunctionArgs(thoughts="To address the issue of the __deepcopy__ method in Django's form fields not properly copying the error messages, we need to locate the implementation of the __deepcopy__ method in the fields.py file and examine the code to understand how it currently handles the error_messages dictionary. This will help us determine the necessary changes to ensure that each form field has its own separate copy of the error_messages dictionary.", file_pattern=None, function_name='__deepcopy__', class_name='FormField'), observation=None, completion=None)

In [41]:
action_step.observation = action.execute(
            action_step.action, new_node.file_context, new_node.workspace
        )
action_step

ActionStep(action=FindFunctionArgs(thoughts="To address the issue of the __deepcopy__ method in Django's form fields not properly copying the error messages, we need to locate the implementation of the __deepcopy__ method in the fields.py file and examine the code to understand how it currently handles the error_messages dictionary. This will help us determine the necessary changes to ensure that each form field has its own separate copy of the error_messages dictionary.", file_pattern=None, function_name='__deepcopy__', class_name='FormField'), observation=Observation(message='No search results found', summary=None, terminal=False, expect_correction=False, properties={'search_hits': [], 'search_tokens': 0, 'fail_reason': 'no_search_hits'}, execution_completion=None), completion=None)

In [42]:
action_step.observation

Observation(message='No search results found', summary=None, terminal=False, expect_correction=False, properties={'search_hits': [], 'search_tokens': 0, 'fail_reason': 'no_search_hits'}, execution_completion=None)

In [50]:
def _execute(agent, node: Node, action_step: ActionStep):
    action = agent._action_map.get(type(action_step.action))
    if not action:
        print(
            f"Node{node.node_id}: Action {node.action.name} not found in action map. "
            f"Available actions: {self._action_map.keys()}"
        )
        raise RuntimeError(f"Action {type(node.action)} not found in action map.")

    try:
        action_step.observation = action.execute(
            action_step.action, node.file_context, node.workspace
        )
        if not action_step.observation:
            print(
                f"Node{node.node_id}: Action {action_step.action.name} returned no observation"
            )
        else:
            node.terminal = action_step.observation.terminal
            if action_step.observation.execution_completion:
                action_step.completion = (
                    action_step.observation.execution_completion
                )

        print(
            f"Executed action: {action_step.action.name}. "
            f"Terminal: {action_step.observation.terminal if node.observation else False}. "
            f"Output: {action_step.observation.message if node.observation else None}"
        )

    except CompletionRejectError as e:
        print(f"Node{node.node_id}: Action rejected: {e.message}")
        action_step.completion = e.last_completion
        action_step.observation = Observation(
            message=e.message,
            is_terminal=True,
        )

In [51]:
for action_step in new_node.action_steps:
    _execute(agent, new_node, action_step)

Could not find module for file docs/howto/deployment/checklist.txt
Could not find module for file docs/howto/deployment/checklist.txt
Could not find module for file docs/howto/deployment/checklist.txt
Could not find module for file docs/ref/contrib/staticfiles.txt
Could not find module for file docs/ref/files/storage.txt
Could not find module for file docs/ref/settings.txt
Content for docs/ref/settings.txt exceeded max_tokens (16000)
Could not find module for file docs/howto/deployment/checklist.txt


Executed action: FindCodeSnippet. Terminal: False. Output: Found the following relevant code:
django/conf/global_settings.py
```

      # ...
   290	FILE_UPLOAD_MAX_MEMORY_SIZE = 2621440  # i.e. 2.5 MB
   291	
   292	# Maximum size in bytes of request data (excluding file uploads) that will be
   293	# read before a SuspiciousOperation (RequestDataTooBig) is raised.
   294	DATA_UPLOAD_MAX_MEMORY_SIZE = 2621440  # i.e. 2.5 MB
   295	
   296	# Maximum number of GET/POST parameters that will be read before a
   297	# SuspiciousOperation (TooManyFieldsSent) is raised.
   298	DATA_UPLOAD_MAX_NUMBER_FIELDS = 1000
   299	
   300	# Directory in which upload streamed files will be temporarily saved. A value of
   301	# `None` will make Django use the operating system's default temporary directory
   302	# (i.e. "/tmp" on *nix systems).
   303	FILE_UPLOAD_TEMP_DIR = None
   304	
   305	# The numeric mode to set newly-uploaded files to. The value should be a mode
   306	# you'd pass directly to o

In [62]:
print(not new_node.is_duplicate and new_node.observation)

message='Found the following relevant code:\ndjango/conf/global_settings.py\n```\n\n      # ...\n   290\tFILE_UPLOAD_MAX_MEMORY_SIZE = 2621440  # i.e. 2.5 MB\n   291\t\n   292\t# Maximum size in bytes of request data (excluding file uploads) that will be\n   293\t# read before a SuspiciousOperation (RequestDataTooBig) is raised.\n   294\tDATA_UPLOAD_MAX_MEMORY_SIZE = 2621440  # i.e. 2.5 MB\n   295\t\n   296\t# Maximum number of GET/POST parameters that will be read before a\n   297\t# SuspiciousOperation (TooManyFieldsSent) is raised.\n   298\tDATA_UPLOAD_MAX_NUMBER_FIELDS = 1000\n   299\t\n   300\t# Directory in which upload streamed files will be temporarily saved. A value of\n   301\t# `None` will make Django use the operating system\'s default temporary directory\n   302\t# (i.e. "/tmp" on *nix systems).\n   303\tFILE_UPLOAD_TEMP_DIR = None\n   304\t\n   305\t# The numeric mode to set newly-uploaded files to. The value should be a mode\n   306\t# you\'d pass directly to os.chmod; s

In [48]:
value_function = ValueFunction(completion_model=completion_model)

### value_function.get_reward函数内部

In [49]:
from litellm.types.llms.openai import ChatCompletionUserMessage
from moatless.value_function.model import Reward

In [50]:
new_messages = message_generator.generate(new_node)

In [51]:
last_message = ""
if new_node.observation.expect_correction and value_function.correction_award is not None:
    last_message += "# Automatic Reward Assessment\n"
    last_message += f"Action expects a correction. Suggested value: {value_function.correction_award}\n\n"
if new_node.action.name == "Finish":
    last_message += "# Completion Reasoning\n"
    last_message += "<reasoning_for_completion>\n"
    last_message += new_node.action.finish_reason
    last_message += "\n</reasoning_for_completion>\n\n"
else:
    last_message += "# Last Executed Action\n"
    last_message += "The following action was executed and its output is the subject of your evaluation:\n\n"
    last_message += "<executed_action>\n"
    last_message += f"Action: {new_node.action.name}\n"
    last_message += new_node.action.to_prompt()
    last_message += "\n## Output\n"
    last_message += new_node.observation.message
    last_message += "\n</executed_action>\n\n"

last_message

'# Last Executed Action\nThe following action was executed and its output is the subject of your evaluation:\n\n<executed_action>\nAction: SemanticSearch\nSearching for code using the query: file upload permissions\n## Output\nFound the following relevant code:\ndjango/conf/global_settings.py\n```\n\n      # ...\n   290\tFILE_UPLOAD_MAX_MEMORY_SIZE = 2621440  # i.e. 2.5 MB\n   291\t\n   292\t# Maximum size in bytes of request data (excluding file uploads) that will be\n   293\t# read before a SuspiciousOperation (RequestDataTooBig) is raised.\n   294\tDATA_UPLOAD_MAX_MEMORY_SIZE = 2621440  # i.e. 2.5 MB\n   295\t\n   296\t# Maximum number of GET/POST parameters that will be read before a\n   297\t# SuspiciousOperation (TooManyFieldsSent) is raised.\n   298\tDATA_UPLOAD_MAX_NUMBER_FIELDS = 1000\n   299\t\n   300\t# Directory in which upload streamed files will be temporarily saved. A value of\n   301\t# `None` will make Django use the operating system\'s default temporary directory\n   

In [52]:
if last_message:
    new_messages.append(
        ChatCompletionUserMessage(role="user", content=last_message)
    )

In [53]:
system_prompt_value = value_function._create_system_prompt(new_node, None)
print(system_prompt_value)

Your role is to evaluate the **last executed action** of the search tree that our AI agents are traversing, to help us determine the best trajectory to solve a programming issue. The agent is responsible for identifying and modifying the correct file(s) in response to the problem statement.

Important: While line numbers may be referenced in the initial problem description, they can shift as changes are made to the file. Focus on whether the agent is modifying the correct logical parts of the code, rather than strictly matching the initially mentioned line numbers. What matters is that the right section of code is being modified, even if its current line number differs from what was originally specified.

At this stage, the agent is still working on the solution. Your task is twofold:
1. **Evaluation**: Assess whether the change done by the **last executed action** is appropriate for addressing the problem and whether the agent is on the right path to resolving the issue. Verify that t

In [55]:
completion_response = value_function.completion_model.create_completion(
    messages=new_messages, system_prompt=system_prompt_value, response_model=Reward
)

In [56]:
completion_response.structured_output

Reward(explanation=None, feedback="An alternative approach could involve searching for existing discussions or issues related to FILE_UPLOAD_PERMISSIONS on Django's GitHub repository or Stack Overflow. This could provide insights into how other developers have addressed similar concerns, and whether any community-driven solutions or workarounds exist. Additionally, exploring the documentation for FILE_UPLOAD_PERMISSIONS could yield more details on its usage and implications.", value=90)

In [57]:
new_node.reward, completion_response = value_function.get_reward(
                    node=new_node
                )

In [58]:
new_node.completions["value_function"] = completion_response

In [59]:
new_node.completions.keys()

dict_keys(['build_action', 'value_function'])

In [ ]:
# def _simulate(node: Node):
#     """Simulate a playout by executing the action and evaluating the result."""

#     if node.observation:
#         print(f"Node{node.node_id}: Action already executed. Skipping.")
#     else:
#         agent.run(node)
#     if self.value_function and not node.is_duplicate and node.observation:
#         try:
#             node.reward, completion_response = self.value_function.get_reward(
#                 node=node
#             )
#             node.completions["value_function"] = completion_response
#             print(f"Node{node.node_id}: The value function returned a reward of {node.reward.value}.")
#         except RejectError as e:
#             self.log(
#                 logger.warning,
#                 f"Node{node.node_id}: Value function rejected: {e.message}",
#             )
#             node.reward = None
#         except RuntimeError as e:
#             self.log(
#                 logger.error,
#                 f"Node{node.node_id}: Value function runtime error: {e.message}",
#             )
#             raise  # Re-raise to abort the entire search

In [ ]:
# _simulate(new_node)

## SearchTree._backpropagate函数内部

In [60]:
reward = new_node.reward.value
reward

90

In [61]:
n = new_node

while n is not None:
    print(n.visits)
    print(n.value)
    n = n.parent

0
None
0
None


In [62]:
def _backpropagate(node: Node):
    """Backpropagate the reward up the tree."""

    if not node.reward:
        print(
            f"Node{node.node_id} has no evaluation. Skipping backpropagation.",
        )
        return

    reward = node.reward.value
    while node is not None:
        node.visits += 1
        if not node.value:
            node.value = reward
        else:
            node.value += reward
        node = node.parent

In [63]:
_backpropagate(new_node)

In [64]:
n = new_node

while n is not None:
    print(n.visits)
    print(n.value)
    n = n.parent

1
90
1
90
